In [1]:
from transformers import AutoTokenizer

# 初始化tokenizer
tokenizer = AutoTokenizer.from_pretrained('Qwen/Qwen2.5-0.5B-Instruct')

from recurrent.interface import AsyncOutput
from recurrent.chat_template.utils import set_chat_template

from recurrent.async_generation_manager import AsyncLLMGenerationManager
class Dummy(AsyncLLMGenerationManager):
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        set_chat_template(tokenizer)

manager = Dummy(
    tokenizer,
)
messages=[
    {"role": "system", "content": "System."},
    {
        "role": "user",
        "content": "User",
    },
    {
        "role": "assistant",
        "content": "Assistant",
    },
    {
        "role": "tool",
        "content": "66666",
    },
    {
        "role": "assistant",
        "content": "777",
    }
]
messages2=[
    {"role": "system", "content": "System."},
    {
        "role": "user",
        "content": "UserUser",
    },
    {
        "role": "assistant",
        "content": "AssistantAssistant",
    },
    {
        "role": "tool",
        "content": "66666",
    },
    {
        "role": "assistant",
        "content": "777777",
        "finished": False
    }
]
import torch


In [2]:
outputs = [AsyncOutput(
        conversations=[messages2],
        sample_index=torch.tensor([0]),
        final_mask=torch.tensor([True]),
        timing_raw={}
    ), AsyncOutput(
        conversations=[messages, messages2],
        sample_index=torch.tensor([1, 1]),
        final_mask=torch.tensor([False, True]),
        timing_raw={}
    )]
outs = [manager.tokenize_output(o) for o in outputs]
def check(out):
    assert len(out['loss_mask'][0]) == len(out['responses'][0])
for out in outs:
    check(out)
outs

[{'prompts': array([array([151644,   8948,    198,   2320,     13, 151645,    271, 151644,
                   872,    198,   1474,   1474, 151645,    198, 151644,  77091,
                   198])                                                       ],
        dtype=object),
  'responses': array([array([151644,   8948,    198,   2610,    525,   1207,  16948,     11,
                  3465,    553,  54364,  14817,     13,   1446,    525,    264,
                 10950,  17847,     13, 151645,    198, 151644,    872,    198,
                  9571, 151645,    198, 151644,  77091,    198,  71703,  71703,
                151645,    198, 151644,    872,    198,     27,  14172,   9655,
                   397,     21,     21,     21,     21,     21,    198,    522,
                 14172,   9655,     29, 151645,    198, 151644,  77091,    198,
                    22,     22,     22,     22,     22,     22])               ],
        dtype=object),
  'loss_mask': array([array([0, 0, 0, 0, 0, 0,

In [3]:
batch = manager.concat_output(
    outs
)
batch

DataProto(batch=TensorDict(
    fields={
        attention_mask: Tensor(shape=torch.Size([3, 49]), device=cpu, dtype=torch.bool, is_shared=False),
        final_mask: Tensor(shape=torch.Size([3]), device=cpu, dtype=torch.bool, is_shared=False),
        input_ids: Tensor(shape=torch.Size([3, 49]), device=cpu, dtype=torch.int64, is_shared=False),
        loss_mask: Tensor(shape=torch.Size([3, 32]), device=cpu, dtype=torch.int64, is_shared=False),
        position_ids: Tensor(shape=torch.Size([3, 49]), device=cpu, dtype=torch.int64, is_shared=False),
        prompts: Tensor(shape=torch.Size([3, 17]), device=cpu, dtype=torch.int64, is_shared=False),
        responses: Tensor(shape=torch.Size([3, 32]), device=cpu, dtype=torch.int64, is_shared=False),
        sample_index: Tensor(shape=torch.Size([3]), device=cpu, dtype=torch.int64, is_shared=False)},
    batch_size=torch.Size([3]),
    device=None,
    is_shared=False), non_tensor_batch={}, meta_info={})

In [4]:
batch.batch

TensorDict(
    fields={
        attention_mask: Tensor(shape=torch.Size([3, 49]), device=cpu, dtype=torch.bool, is_shared=False),
        final_mask: Tensor(shape=torch.Size([3]), device=cpu, dtype=torch.bool, is_shared=False),
        input_ids: Tensor(shape=torch.Size([3, 49]), device=cpu, dtype=torch.int64, is_shared=False),
        loss_mask: Tensor(shape=torch.Size([3, 32]), device=cpu, dtype=torch.int64, is_shared=False),
        position_ids: Tensor(shape=torch.Size([3, 49]), device=cpu, dtype=torch.int64, is_shared=False),
        prompts: Tensor(shape=torch.Size([3, 17]), device=cpu, dtype=torch.int64, is_shared=False),
        responses: Tensor(shape=torch.Size([3, 32]), device=cpu, dtype=torch.int64, is_shared=False),
        sample_index: Tensor(shape=torch.Size([3]), device=cpu, dtype=torch.int64, is_shared=False)},
    batch_size=torch.Size([3]),
    device=None,
    is_shared=False)

In [5]:
batch.batch['prompts'].shape

torch.Size([3, 17])

In [6]:
from recurrent.utils import pad_tensor_list_to_length
length = [1,2,1]
pad_tensor_list_to_length(
    [torch.arange(l) for l in length],
    pad_token_id=0,
    return_mask=True,
)

(tensor([[0, 0],
         [0, 1],
         [0, 0]]),
 tensor([[False,  True],
         [ True,  True],
         [False,  True]]))

In [7]:
for k in batch.batch.keys():
    print(k)
    if k  in ['prompts', 'responses', 'input_ids']:
        for l in tokenizer.batch_decode(batch.batch[k]):
            print(repr(l))
    else:
        print(batch.batch[k])
    

prompts
'<|im_start|>system\nSystem.<|im_end|>\n\n<|im_start|>user\nUserUser<|im_end|>\n<|im_start|>assistant\n'
'<|endoftext|><|im_start|>system\nSystem.<|im_end|>\n\n<|im_start|>user\nUser<|im_end|>\n<|im_start|>assistant\n'
'<|im_start|>system\nSystem.<|im_end|>\n\n<|im_start|>user\nUserUser<|im_end|>\n<|im_start|>assistant\n'
responses
'AssistantAssistant<|im_end|>\n<|im_start|>user\n<tool_response>\n66666\n</tool_response><|im_end|>\n<|im_start|>assistant\n777777'
'Assistant<|im_end|>\n<|im_start|>user\n<tool_response>\n66666\n</tool_response><|im_end|>\n<|im_start|>assistant\n777<|im_end|><|endoftext|><|endoftext|><|endoftext|>'
'AssistantAssistant<|im_end|>\n<|im_start|>user\n<tool_response>\n66666\n</tool_response><|im_end|>\n<|im_start|>assistant\n777777'
input_ids
'<|im_start|>system\nSystem.<|im_end|>\n\n<|im_start|>user\nUserUser<|im_end|>\n<|im_start|>assistant\nAssistantAssistant<|im_end|>\n<|im_start|>user\n<tool_response>\n66666\n</tool_response><|im_end|>\n<|im_start|>